In [1]:
# !pip install grpcio grpcio-tools

# Scikit-Learn Iris Model using customData

* Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
    * Run locally on Docker to test
    * Deploy on seldon-core running on a Kubernetes cluster

## Dependencies

* [s2i](https://github.com/openshift/source-to-image)
* Seldon Core v1.0.3+ installed
* `pip install sklearn seldon-core protobuf grpcio`

## Train locally

In [2]:
import os

import numpy as np
from sklearn import datasets
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


def main():
    clf = LogisticRegression()
    p = Pipeline([("clf", clf)])
    print("Training model...")
    p.fit(X, y)
    print("Model trained!")

    filename_p = "IrisClassifier.sav"
    print("Saving model in %s" % filename_p)
    joblib.dump(p, filename_p)
    print("Model saved!")


if __name__ == "__main__":
    print("Loading iris data set...")
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print("Dataset loaded!")
    main()

Loading iris data set...
Dataset loaded!
Training model...
Model trained!
Saving model in IrisClassifier.sav
Model saved!


/home/kachau/.local/share/virtualenvs/seldon-core-zr8TvrYf/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Custom Protobuf Specification

First, we'll need to define our custom protobuf specification so that it can be leveraged.

In [3]:
%%writefile iris.proto

syntax = "proto3";

package iris;

message IrisPredictRequest {
    float sepal_length = 1;
    float sepal_width = 2;
    float petal_length = 3;
    float petal_width = 4;
}

message IrisPredictResponse {
    float setosa = 1;
    float versicolor = 2;
    float virginica = 3;
}

Overwriting iris.proto


## Custom Protobuf Compilation

We will need to compile our custom protobuf for python so that we can unpack the `customData` field passed to our `predict` method later on.

In [4]:
!python -m grpc.tools.protoc --python_out=./ --proto_path=. iris.proto

## gRPC test

Wrap model using s2i

In [8]:
!s2i build  ../ seldonio/seldon-core-s2i-python38-ubi8:1.11.0 chauhankaranraj/sklearn-iris-customdata:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Created wheel for PTable: filename=PTable-0.9.2-py3-none-any.whl size=22907 sha256=8a39e888207039abb8d1c1ad720b90a2cbc94759fea195251be6f37988beda97
Stored in directory: /root/.cache/pip/wheels/1b/3a/02/8d8da2bca2223dda2f827949c88b2d82dc85dccbc2bb6265e5
Successfully built PTable
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
created path: ./licenses/license_info.csv
created path: ./licenses/license.txt
Build completed successfully


Serve the model locally

In [10]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 chauhankaranraj/sklearn-iris-customdata:0.1

0bd97e43d7381a5647d64547174977ef04f9f113a84d56ac2432b3aab0b47c6d


Test using custom protobuf payload

In [11]:
import grpc
from iris_pb2 import IrisPredictRequest, IrisPredictResponse

from seldon_core.proto import prediction_pb2, prediction_pb2_grpc

channel = grpc.insecure_channel("localhost:5000")
stub = prediction_pb2_grpc.ModelStub(channel)

iris_request = IrisPredictRequest(
    sepal_length=7.233, sepal_width=4.652, petal_length=7.39, petal_width=0.324
)

seldon_request = prediction_pb2.SeldonMessage()
seldon_request.customData.Pack(iris_request)

response = stub.Predict(seldon_request)

iris_response = IrisPredictResponse()
response.customData.Unpack(iris_response)

print(iris_response)

ModuleNotFoundError: No module named 'seldon_core'

Stop serving model

In [12]:
!docker rm iris_predictor --force

Error: No such container: iris_predictor


## Setup Seldon Core

Use the [setup notebook](https://github.com/SeldonIO/seldon-core/blob/master/notebooks/seldon_core_setup.ipynb) to setup Seldon Core with an ingress - either Ambassador or Istio

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

* Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080`
* Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80`

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

## Deploy your Seldon Model

We first create a configuration file:

In [ ]:
%%writefile sklearn_iris_customdata_deployment.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-deployment-example
spec:
  name: sklearn-iris-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: groszewn/sklearn-iris-customdata:0.1
          imagePullPolicy: IfNotPresent
          name: sklearn-iris-classifier
    graph:
      children: []
      endpoint:
        type: GRPC
      name: sklearn-iris-classifier
      type: MODEL
    name: sklearn-iris-predictor
    replicas: 1

### Run the model in our cluster

Apply the Seldon Deployment configuration file we just created

In [ ]:
!kubectl create -f sklearn_iris_customdata_deployment.yaml

### Check that the model has been deployed

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-deployment-example -o jsonpath='{.items[0].metadata.name}')

## Test by sending prediction calls

`IrisPredictRequest` sent via the `customData` field.

In [ ]:
iris_request = IrisPredictRequest(
    sepal_length=7.233, sepal_width=4.652, petal_length=7.39, petal_width=0.324
)

seldon_request = prediction_pb2.SeldonMessage()
seldon_request.customData.Pack(iris_request)

channel = grpc.insecure_channel("localhost:8003")
stub = prediction_pb2_grpc.SeldonStub(channel)

metadata = [("seldon", "seldon-deployment-example"), ("namespace", "seldon")]

response = stub.Predict(request=seldon_request, metadata=metadata)

iris_response = IrisPredictResponse()
response.customData.Unpack(iris_response)

print(iris_response)

### Cleanup our deployment

In [ ]:
!kubectl delete -f sklearn_iris_customdata_deployment.yaml